# 585.683 Introduction to Brain Computer Interfaces
## Mod 07 - Brain-Computer Interfacing – Introduction to Brain Computer Interfaces

### Date Submitted: July, DAY, 2022

### Eitan Waks

Answer the following questions using the robintibor/high-gamma-dataset – specifically data/train/1.mat.  Make sure you provide a notebook or source code as an appendix to this assignment.  Again, make sure all plots have labeled axes and meaningful titles.

Hint: You should be able to copy paste and re-use sections from the lecture notebooks provided in this module to complete this assignment with very little coding.

#### Q1.  Preprocess the dataset in one continuous stretch:
##### 1.	Apply a common average reference using all EEG channels (yes, even bad ones if they exist)
##### 2.	For simplicity, don’t decimate the data unless you want to or have to due to memory constraints
##### 3.	Bandpass filter the data between 1.0 and 30 Hz

##### Then plot all channels whose name contains the string “AF” (hint: there’s 11) for two stretches of data.  A 10 second stretch beginning at 200 seconds into the file, and a 10 second stretch starting at 600 seconds into the file. [4 pts each]

##### Which one of these plots contains an artifact, and what is the most likely source of the artifact? [2 pts]

In [4]:
# imports
import mne
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import braindecode
from braindecode.datasets.bbci import  BBCIDataset


In [5]:
# Paths
dataset_name = '1'
dataset_filetype = 'edf'
repository = 'BCI_MOD7_HW'
app_dir = Path('/', 'app')
data_dir = app_dir.joinpath('data')
notebook_dir = app_dir.joinpath(repository)
data_file = data_dir.joinpath(f'{dataset_name}.{dataset_filetype}')

In [ ]:
# Load the dataset
chs_EOG = ['EOG EOGh', 'EOG EOGv']
chs_EMG = ['EMG EMG_RH', 'EMG EMG_LH', 'EMG EMG_RF']
raw = mne.io.read_raw_edf(data_file, eog=chs_EOG, misc=chs_EMG, stim_channel='auto', exclude=(), infer_types=False, preload=True, verbose=None)
original_raw = raw.copy()
raw

In [ ]:
# re-reference to the common average using MNE
raw_car = raw.copy().set_eeg_reference( 'average' );

In [ ]:
# Bandpass filter
lowpass_freq = 1
highpass_freq = 30
raw_filtered = raw_car.copy().filter(l_freq=lowpass_freq, h_freq=highpass_freq)

In [ ]:
# Set list of all channels whose name contains the string “AF” and the corresponding indeces
AF_ch = [ch for ch in raw.ch_names if 'AF' in ch]
AF_ch_idxs = [raw.ch_names.index(ch) for ch in AF_ch]
raw_filtered_AF = raw_filtered.pick(AF_ch)  # selects only the EEG channels

In [ ]:
# plot all channels whose name contains the string “AF”. 
# A 10 second stretch beginning at 200 seconds into the file
raw_filtered_AF.plot(start=200, duration=10)

In [ ]:
# plot all channels whose name contains the string “AF”. 
# A 10 second stretch starting at 600 seconds into the file.
raw_filtered_AF.plot(start=600, duration=10)

##### The plot which begins at 600 seconds contains in artifact. The most likely cause for this artifact is an eye blink.

#### Q2. Following the preprocessing previously applied in Q1, segment trials from -1.0 to +3.0 seconds around the event timestamps in the dataset events structure, and plot the trial-average (mean) timeseries for “Cz”, “C3”, “C4, “Pz”, and “Oz”, without regard to trial type (i.e., including all trial types) [2 pts per plot.]

In [ ]:
# Set list of channels and corresponding indeces
chs = ['EEG Cz', 'EEG C3', 'EEG C4', 'EEG Pz', 'EEG Oz']
ch_idxs = [raw.ch_names.index(ch) for ch in chs]

In [ ]:
# Get events from the database
annotations = raw.annotations
events_from_annot, event_dict = mne.events_from_annotations(raw)
print(event_dict)